In [1]:
# | default_exp backend.tools.fs_read

https://docs.aws.amazon.com/amazonq/latest/qdeveloper-ug/git-aware-selection.html

In [1]:
# | export

import os
import re
import time
import logging
from pathlib import Path
from typing import Dict, List, Any, Optional, Literal
from fuzzywuzzy import fuzz
import fnmatch
import json
from concurrent.futures import ThreadPoolExecutor, as_completed
from pydantic import BaseModel, field_validator, Field, ValidationInfo
from enum import Enum

try:
    import git
except ImportError:
    git = None

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ToolCallMode(str, Enum):
    DISCOVER = "discover"
    EXTRACT = "extract"

class FsReadOperation(BaseModel):
    mode: ToolCallMode
    path: str = Field(..., description="Specific file or directory path to operate on, e.g., '/project/src' or 'app.py'. Mandatory for precision; scans from here. Defaults to current working directory (project root) if empty, but always specify to avoid broad scans.")
    query: Optional[str] = Field(None, description="Search query: For DISCOVER, fuzzy-matches filenames/content for relevance; for EXTRACT, regex or substring to find matches (regex default). Required for EXTRACT; optional for DISCOVER (glob-only if absent).")
    file_pattern: str = Field("*.py|*.go|*.rs|*.java|*.js|*.ts|*.cpp|*.c|*.h|*.cs", description="Glob filter for files, e.g., '*.py' or '*.go|*.rs'. Defaults to common source file extensions (*.py|*.go|*.rs|*.java|*.js|*.ts|*.cpp|*.c|*.h|*.cs) in EXTRACT mode to process code files; '*' in DISCOVER mode for broader searches. Specify to optimize scans.")
    max_depth: Optional[int] = Field(10, description="Max recursion depth for DISCOVER in directories (default 10); set lower (e.g., 3) for flat projects to skip deep dirs like node_modules.")
    context_lines: Optional[int] = Field(3, description="Lines to include before/after each match in EXTRACT (default 3); e.g., 5 for full function context in code.")
    regex_mode: Optional[bool] = Field(True, description="True (default) treats query as regex in EXTRACT, e.g., r'def\\s+login'; false for simple text search.")
    start_line: Optional[int] = Field(1, description="Fallback start line for EXTRACT if no matches (default 1); for partial file reads.")
    end_line: Optional[int] = Field(-1, description="Fallback end line for EXTRACT (-1 = file end); combines with start_line for targeted reads.")
    max_files: Optional[int] = Field(100, description="Max files to scan in DISCOVER (default 100); caps computation in large repos for <5s response.")
    limit_results: Optional[int] = Field(5, description="Max results from DISCOVER (top N by score, default 5); keeps outputs lean for tools like code_interpreter.")

    @field_validator("path")
    @classmethod
    def validate_path(cls, value: str) -> str:
        if not value:
            value = os.getcwd()
            logger.info(f"Path empty; defaulting to project root: {value}")
        path_obj = Path(value)
        if not path_obj.exists():
            logger.warning(f"Path {value} does not exist; returning empty results.")
        return str(path_obj)

    @field_validator("query")
    @classmethod
    def validate_query(cls, value: Any, info: ValidationInfo) -> Any:
        mode = info.data.get("mode")
        if mode == ToolCallMode.EXTRACT and value is None:
            raise ValueError("Query is required for extract mode to perform regex or substring matching.")
        if mode == ToolCallMode.DISCOVER and value is None:
            return ""
        return value

    @field_validator("file_pattern")
    @classmethod
    def validate_file_pattern(cls, value: Any, info: ValidationInfo) -> Any:
        mode = info.data.get("mode")
        if value is None:
            return "*.py|*.txt|*.md|*.yaml|*.json" if mode == ToolCallMode.EXTRACT else "*"
        if not isinstance(value, str) or ('*' not in value and '?' not in value and '|' not in value):
            raise ValueError("file_pattern must be a valid glob, e.g., '*.py' or '*.py|*.txt'")
        return value

class FsReadParams(BaseModel):
    operations: List[FsReadOperation]

class FsReadTool:
    def __init__(self):
        self.repo = None
        self.gitignore_patterns = []
        self.git_status_cache = {}
        self.query_cache = {}
        self.common_exclusions = [
            ".*", "*.pyc", "*.o", "*.obj", "*.class", "*.exe", "*.dll", "*.so",
            "*.lock", "node_modules/*", "dist/*", "build/*", "__pycache__/*",
            "*.bin", "*.zip", "*.tar.gz", "*.log"
        ]
        if git:
            try:
                self.repo = git.Repo(os.getcwd(), search_parent_directories=True)
                self.gitignore_patterns = self._load_gitignore()
                self._batch_load_git_status()
            except (git.InvalidGitRepositoryError, git.NoSuchPathError):
                pass

    def _load_gitignore(self) -> List[str]:
        """Load .gitignore patterns recursively from repo root and subdirs."""
        patterns = self.common_exclusions.copy()
        try:
            repo_root = Path(self.repo.working_dir)
            for gitignore in repo_root.rglob(".gitignore"):
                try:
                    with open(gitignore, 'r', encoding='utf-8', errors='ignore') as f:
                        for line in f:
                            line = line.strip()
                            if line and not line.startswith('#'):
                                rel_path = str(gitignore.parent.relative_to(repo_root)) + '/' + line
                                if rel_path.startswith('./'):
                                    rel_path = rel_path[2:]
                                patterns.append(rel_path)
                except (OSError, UnicodeDecodeError):
                    logger.debug(f"Failed to read {gitignore}")
        except (OSError, Exception):
            logger.warning("Failed to load .gitignore patterns; using common exclusions only.")
        return patterns

    def _batch_load_git_status(self):
        """Batch load Git status for all files."""
        try:
            status_output = self.repo.git.status('--porcelain', '-u', 'all')
            for line in status_output.split('\n'):
                if line.strip():
                    parts = line.split(maxsplit=1)
                    if len(parts) >= 2:
                        status_code = parts[0][0]
                        rel_path = parts[1].strip()
                        self.git_status_cache[rel_path] = status_code if status_code != '?' else 'untracked'
        except (OSError, Exception):
            logger.warning("Failed to batch-load Git status; falling back to per-file.")

    def _get_file_info(self, path: str, lightweight: bool = True) -> Dict[str, Any]:
        """Optimized file info with exclusions."""
        path_obj = Path(path)
        if not path_obj.exists():
            return {"error": "File not found"}

        try:
            stat = path_obj.stat()
        except OSError:
            return {"error": "Stat failed"}

        rel_path = path_obj.name
        if self.repo:
            try:
                rel_path = str(path_obj.relative_to(self.repo.working_dir))
            except ValueError:
                pass
        if any(fnmatch.fnmatch(rel_path, p) for p in self.gitignore_patterns):
            return {"error": "Excluded by .gitignore or common patterns"}

        file_size = stat.st_size
        mtime = stat.st_mtime
        recency_boost = 1.0 if time.time() - mtime < 86400 else 0.5

        file_info = {
            "size": file_size,
            "is_binary": False,
            "file_type": path_obj.suffix.lower(),
            "is_large": file_size > 1024 * 1024,
            "mtime": mtime,
            "recency_boost": recency_boost,
            "git_status": None,
        }

        if self.repo:
            rel_path = os.path.relpath(path, self.repo.working_dir)
            file_info["git_status"] = self.git_status_cache.get(rel_path, "clean")
            if file_info["git_status"] in ["M", "A", "?"]:
                file_info["recency_boost"] = 1.0

        if lightweight:
            return file_info

        try:
            with open(path, 'rb') as f:
                chunk = f.read(512)
                if b'\x00' in chunk:
                    file_info["is_binary"] = True
                    return file_info
        except (OSError, UnicodeDecodeError):
            file_info["is_binary"] = True
            return file_info

        try:
            if file_size < 10 * 1024 * 1024:
                with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                    file_info["lines"] = sum(1 for _ in f)
            else:
                with open(path, 'r', encoding='utf-8', errors='ignore') as f:
                    sample_lines = sum(1 for _, _ in zip(range(10000), f))
                    file_info["lines"] = int((file_size / (f.tell() or 1)) * sample_lines)
        except (OSError, UnicodeDecodeError):
            pass

        return file_info

    def _score_file_content(self, file_path: str, query_lower: str) -> float:
        """Parallel-friendly content scorer."""
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read(1024).lower()
            return fuzz.partial_ratio(query_lower, content)
        except (OSError, UnicodeDecodeError):
            return 0.0
    
    def _discover_files(self, path: str, file_pattern: str, query: str, max_depth: int, max_files: int = 100, limit_results: int = 5, timeout: float = 10.0) -> List[Dict]:
        """Blazing-fast discovery with scandir, parallel scoring, and exclusions."""
        start_time = time.time()
        path_obj = Path(path)
        if not path_obj.is_dir():
            return [{"error": f"Path {path} is not a directory for discovery"}]
    
        query_lower = self.query_cache.get(query, query.lower() if query else "")
        if query and query not in self.query_cache:
            self.query_cache[query] = query_lower
    
        candidates = []
        # Support multiple patterns (e.g., *.py|*.txt)
        patterns = file_pattern.split('|')
    
        def _collect_with_scandir(dir_path: str, current_depth: int, file_count: List[int]) -> None:
            if current_depth > max_depth or file_count[0] >= max_files or time.time() - start_time > timeout:
                return
            try:
                with os.scandir(dir_path) as entries:
                    for entry in entries:
                        if time.time() - start_time > timeout:
                            logger.warning(f"Discovery timeout after {timeout}s")
                            return
                        rel_path = entry.name
                        if self.repo:
                            try:
                                rel_path = str(Path(entry.path).relative_to(self.repo.working_dir))
                            except ValueError:
                                pass
                        if any(fnmatch.fnmatch(rel_path, p) for p in self.gitignore_patterns):
                            continue
                        if any(fnmatch.fnmatch(entry.name, p) for p in patterns) and entry.is_file() and Path(entry.path).exists():
                            file_info = self._get_file_info(entry.path, lightweight=True)
                            if "error" in file_info or file_info.get("is_binary"):
                                continue
                            base_score = 0
                            if query:
                                name_score = fuzz.partial_ratio(query_lower, entry.name.lower())
                                base_score = name_score
                                if name_score < 50 and not file_info.get("is_binary"):
                                    base_score = max(base_score, self._score_file_content(entry.path, query_lower))
                            git_boost = 20 if file_info.get("git_status") in ["M", "A"] else 10 if file_info.get("git_status") == "untracked" else 0
                            mtime_boost = 10 if file_info["recency_boost"] > 0.8 else 0
                            candidates.append({
                                "path": entry.path,
                                "name": entry.name,
                                "type": "file",
                                "score": base_score + git_boost + mtime_boost,
                                "file_info": file_info
                            })
                            file_count[0] += 1
                            if file_count[0] >= max_files:
                                logger.info(f"Reached max files: {max_files}")
                                return
                        if entry.is_dir():
                            _collect_with_scandir(entry.path, current_depth + 1, file_count)
            except (OSError, PermissionError) as e:
                logger.warning(f"Scan error in {dir_path}: {e}")
    
        file_count = [0]
        _collect_with_scandir(str(path_obj), 0, file_count)
    
        if not candidates:
            return [{"error": "No files matched the criteria"}]
    
        if query and len(candidates) > 10:
            low_scorers = [c for c in candidates if c["score"] < 50 and not c["file_info"].get("is_binary")]
            if low_scorers:
                with ThreadPoolExecutor(max_workers=4) as executor:
                    futures = {executor.submit(self._score_file_content, c["path"], query_lower): c for c in low_scorers[:20]}
                    for future in as_completed(futures):
                        content_score = future.result()
                        if content_score > 0:
                            idx = low_scorers.index(futures[future])
                            low_scorers[idx]["score"] = max(low_scorers[idx]["score"], content_score)
    
        candidates.sort(key=lambda x: x["score"], reverse=True)
        found = []
        for item in candidates[:limit_results]:
            found.append({
                "path": item["path"],
                "name": item["name"],
                "type": item["type"],
                "score": item["score"],
                "git_status": item["file_info"].get("git_status", "N/A"),
                "recency": item["file_info"]["mtime"]
            })
    
        logger.info(f"Discovered {len(found)} files in {time.time() - start_time:.2f}s (processed {file_count[0]})")
        return found
    
    def _extract_content(self, path: str, query: str, file_pattern: str, context_lines: int, regex_mode: bool, start_line: int, end_line: int) -> str:
        """Context-aware extraction with byte cap."""
        path_obj = Path(path)
        if not path_obj.exists():
            return json.dumps({"error": f"Path {path} does not exist"})
    
        if path_obj.is_dir():
            snippets = []
            discovered = self._discover_files(path, file_pattern, query, 1, max_files=10, limit_results=10)
            for file_item in discovered:
                if "error" not in file_item and isinstance(file_item.get("path"), str) and Path(file_item["path"]).is_file():
                    file_info = self._get_file_info(file_item["path"], lightweight=False)
                    if "error" in file_info or file_info.get("is_binary"):
                        continue
                    file_snip = self._extract_from_file(file_item["path"], query, context_lines, regex_mode, start_line, end_line)
                    if file_snip and not file_snip.startswith("[Binary or invalid file") and not file_snip.startswith("Error reading file"):
                        snippets.append({"file": file_item["path"], "snippet": file_snip})
            if not snippets:
                return json.dumps({"error": f"No valid text files matched the criteria in {path}"})
            content = json.dumps(snippets)
        else:
            content = self._extract_from_file(str(path_obj), query, context_lines, regex_mode, start_line, end_line)
    
        if len(content) > 16 * 1024:
            content = content[:16 * 1024] + "\n... [truncated]"
        return content
    
    def _extract_from_file(self, file_path: str, query: str, context_lines: int, regex_mode: bool, start_line: int, end_line: int) -> str:
        """Line-by-line extraction with byte cap."""
        if not isinstance(file_path, str) or not Path(file_path).is_file():
            return f"[Invalid file path: {file_path}]"
    
        file_info = self._get_file_info(file_path, lightweight=False)
        if "error" in file_info or file_info.get("is_binary"):
            return f"[Binary or invalid file: {file_info.get('size', 0)} bytes]"
    
        try:
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                matches = []
                pattern = None
                if query and regex_mode:
                    pattern = re.compile(query, re.IGNORECASE | re.MULTILINE)
                elif query:
                    query_lower = query.lower()
    
                current_line = start_line
                byte_count = 0
                line_buffer = []
                for line_num, line in enumerate(f, start_line):
                    if current_line > end_line and end_line != -1:
                        break
                    line_stripped = line.rstrip()
                    byte_count += len(line_stripped.encode('utf-8'))
                    if byte_count > 16 * 1024:
                        break
                    line_buffer.append((current_line, line_stripped))
                    if len(line_buffer) > 2 * context_lines + 1:
                        line_buffer.pop(0)
    
                    matched = False
                    if pattern:
                        if pattern.search(line):
                            matched = True
                    elif query:
                        if query_lower in line.lower():
                            matched = True
    
                    if matched:
                        context_start = max(1, current_line - context_lines)
                        context_end = current_line + context_lines + 1
                        context_lines_content = [f"Line {ln}: {text}" for ln, text in line_buffer if context_start <= ln < context_end]
                        matches.append({
                            "line_number": current_line,
                            "content": line_stripped,
                            "context": "\n".join(context_lines_content)
                        })
                    current_line += 1
    
                snippets = []
                for match in matches:
                    snippet = f"Line {match['line_number']}: {match['content']}\n{match['context']}\n"
                    snippets.append(snippet)
                content = '\n--- Match ---\n'.join(snippets) if matches else "No matches found; fallback content truncated."
        except (OSError, UnicodeDecodeError) as e:
            return f"Error reading file: {str(e)}"
    
        git_info = file_info['git_status']
        if not git_info:
            git_info = f"N/A (non-Git: mtime {file_info['mtime']})"
        
        metadata = (
            f"\n--- File Info: {file_info['size']} bytes, "
            f"{file_info.get('lines', 0)} lines, Git: {git_info} ---"
        )
        return content + metadata

    def get_tool_schema(self) -> Dict[str, Any]:
        """Detailed schema for LLMs."""
        return {
            "type": "function",
            "function": {
                "name": "fs_read",
                "description": "Blazing-fast tool for discovering relevant files (fuzzy/glob, respects .gitignore and excludes binaries/lock files) and extracting context-aware snippets (regex with surrounding lines). Optimized for <5s response in large repos; uses Git or mtime for prioritization. Ideal for chaining with fs_write, code_interpreter, or task_executor.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "operations": {
                            "type": "array",
                            "items": {
                                "type": "object",
                                "properties": {
                                    "mode": {
                                        "type": "string",
                                        "enum": [mode.value for mode in ToolCallMode],
                                        "description": "Select 'discover' to find/list top relevant files (fuzzy if query given, else glob-based); 'extract' to pull content snippets with context (regex or substring matches)."
                                    },
                                    "path": {
                                        "type": "string",
                                        "description": "Mandatory file or directory path, e.g., '/project/src' or 'app.py'. Scans from here; defaults to project root if empty (avoid this for precision)."
                                    },
                                    "query": {
                                        "type": "string",
                                        "description": "Search term: In DISCOVER, fuzzy-matches filenames/content; in EXTRACT, regex (e.g., r'def\\s+login') or substring. Required for EXTRACT; optional for DISCOVER."
                                    },
                                    "file_pattern": {
                                        "type": "string",
                                        "description": "Glob filter, e.g., '*.py' or 'config*.yaml'. Defaults to '*' (all files); use to focus scans."
                                    },
                                    "max_depth": {
                                        "type": "integer",
                                        "description": "Max dir recursion for DISCOVER (default 10); lower (e.g., 3) for flat projects to skip node_modules."
                                    },
                                    "context_lines": {
                                        "type": "integer",
                                        "description": "Lines around each match in EXTRACT (default 3); e.g., 5 for full code blocks."
                                    },
                                    "regex_mode": {
                                        "type": "boolean",
                                        "description": "True (default) treats query as regex in EXTRACT; false for simple text search."
                                    },
                                    "start_line": {
                                        "type": "integer",
                                        "description": "Fallback start line for EXTRACT if no matches (default 1)."
                                    },
                                    "end_line": {
                                        "type": "integer",
                                        "description": "Fallback end line for EXTRACT (-1 = file end)."
                                    },
                                    "max_files": {
                                        "type": "integer",
                                        "description": "Max files to scan in DISCOVER (default 100); ensures <5s response."
                                    },
                                    "limit_results": {
                                        "type": "integer",
                                        "description": "Max results from DISCOVER (default 5); keeps outputs lean."
                                    }
                                },
                                "required": ["mode", "path"]
                            }
                        }
                    },
                    "required": ["operations"]
                }
            }
        }

    def execute(self, params: FsReadParams) -> Dict[str, Any]:
        """Execute with chainable JSON output."""
        results = []
        git_detected = self.repo is not None
        total_processed = 0

        for op in params.operations:
            mode = op.mode.value
            path = op.path
            output_data = None
            error = None

            try:
                if mode == "discover":
                    output_data = self._discover_files(path, op.file_pattern, op.query or "", op.max_depth, op.max_files, op.limit_results)
                    total_processed += len([x for x in output_data if "error" not in x])
                elif mode == "extract":
                    output_data = self._extract_content(path, op.query, op.file_pattern, op.context_lines, op.regex_mode, op.start_line, op.end_line)
                    total_processed += 1
            except (OSError, UnicodeDecodeError, Exception) as e:
                error = {"type": "ExecutionError", "message": str(e)}
                logger.error(f"Error in {mode} on {path}: {str(e)}")

            results.append({
                "mode": mode,
                "path": path,
                "data": output_data,
                "error": error
            })

        return {
            "success": all(r.get("error") is None for r in results),
            "data": results,
            "error": None if all(r.get("error") is None for r in results) else {"type": "BatchError", "message": "Some operations failed—check individual errors."},
            "metadata": {
                "git_detected": git_detected,
                "processed_files": total_processed,
                "description": "Results ready for chaining: Use the returned 'data[].path' and 'data[].snippet' fields as inputs for the next processing step."
                
            }
        }

In [2]:
# | hide
tool = FsReadTool()
params = FsReadParams(operations=[
    FsReadOperation(mode=ToolCallMode.DISCOVER, path="/home/pranav-pc/projects/applied-GenAI-lab", query="system prompt", file_pattern="*.py"),
    FsReadOperation(mode=ToolCallMode.EXTRACT, path="/home/pranav-pc/projects/applied-GenAI-lab/", query=r"def\s+get_system_prompt", regex_mode=True, context_lines=5)
])
result = tool.execute(params)
print(json.dumps(result, indent=2))  # Chainable JSON for LLM

NameError: name 'FsReadTool' is not defined